# Importando as bibliotecas

In [ ]:
import pandas as pd
from db import connection_db as conndb
from db import filters
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

tqdm.pandas()
import numpy as np

Conectando ao banco de dados do MongoDB

In [ ]:
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
col_name = 'dadosSemFiltros'

mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_name)
mongo_connection.connect()
collection = mongo_connection.collection

Aplicando pipeline para preparação dos dados

In [ ]:
collection_filters = filters.CollectionFilters(collection)

# Aplicando pipeline 1
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')

# Aplicando pipeline 2 - pega os posts que tenha a data de publicação de no máximo 7 anos e no mínimo 2 anos
collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')

# Aplicando pipeline 3 - desenrola os posts em documentos individuais e cria novas colunas
collection_filters.apply_pipeline3('postsComBDIAndInfos')

# Aplicando pipeline 4 - aplica o filtro para selecionar apenas os posts que tem uma data de publicação válida
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')

collection = collection_filters.collection

In [ ]:
documentos = collection.find()
df_original = pd.DataFrame(list(documentos))

In [ ]:
# Visualizar as primeiras linhas do dataframe
df_original.head()

In [ ]:
# Visualizar as ultimas linhas do dataframe
df_original.tail()

In [ ]:
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df


# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

# Aplicando a função
df_original = preencher_e_converter(df_original, colunas_para_converter)

# Filtragem dos dados
df_original.drop(columns=['_id', 'diaDaSemana', 'hora', 'minutos', 'diaDoMes', 'mes', 'ano'], inplace=True)

# Criando novas colunas com o pandas
df_original['data'] = df_original['postCreatedTime'].dt.date
df_original['data'] = pd.to_datetime(df_original['data'])
df_original['mes'] = df_original['data'].dt.to_period('M')
df_original['semana'] = df_original['data'].dt.to_period('W')

In [ ]:
# Filtrar posts do ano de 2016
df_2017 = df_original[df_original['data'].dt.year == 2017]
df_2017 = df_2017[~((df_2017['data'].dt.month == 12) & (df_2017['data'].dt.year == 2017))]
df_2017 = df_2017[df_2017['data'].dt.month >= 5]
df_2017 = df_2017[df_2017['suicida'] == 3]
df_2017 = df_2017.copy()
df_2017.head()

In [ ]:
# Remover linhas onde ambas as colunas 'postMessage' e 'postStory' estão vazias ou nulas
df_2017 = df_2017.dropna(subset=['postMessage', 'postStory'], how='all')  # Remove quando ambas são NaN
df_2017 = df_2017[~((df_2017['postMessage'].str.strip() == '') & (
        df_2017['postStory'].str.strip() == ''))]  # Remove quando ambas são strings vazias

df_2017 = df_2017.copy()

In [ ]:
# Calcular a contagem de postagens por usuário
post_counts = df_2017.groupby('id_usuario').size().reset_index(name='post_count')

# Visualizar a distribuição
plt.hist(post_counts['post_count'], bins=50)
plt.xlabel('Número de Postagens')
plt.ylabel('Quantidade de Usuários')
plt.title('Distribuição de Postagens por Usuário')
plt.show()

In [ ]:
# Definir os limites superior e inferior
limite_superior = post_counts['post_count'].quantile(0.85)
limite_inferior = post_counts['post_count'].quantile(0.15)

# Filtrar usuários com alta atividade
usuarios_alta_ativ = post_counts[post_counts['post_count'] > limite_superior]['id_usuario']

# Filtrar usuários com baixa atividade
usuarios_baixa_ativ = post_counts[post_counts['post_count'] < limite_inferior]['id_usuario']

# Exibir o número de usuários filtrados
print(f'Número de usuários com alta atividade: {len(usuarios_alta_ativ)}')
print(f'Número de usuários com baixa atividade: {len(usuarios_baixa_ativ)}')

In [ ]:
df_normal = df_2017[~(df_2017['id_usuario'].isin(usuarios_baixa_ativ))].copy()
norm_post_count = df_normal.groupby('id_usuario').size().reset_index(name='post_count')

df_alta_ativ = df_2017[df_2017['id_usuario'].isin(usuarios_alta_ativ)].copy()
alt_post_count = df_alta_ativ.groupby('id_usuario').size().reset_index(name='post_count')

df_baixa_ativ = df_2017[df_2017['id_usuario'].isin(usuarios_baixa_ativ)].copy()
baixa_post_count = df_baixa_ativ.groupby('id_usuario').size().reset_index(name='post_count')

In [ ]:
def plotar_distribuicao(post_count_df, titulo):
    plt.hist(post_count_df['post_count'], bins=50)
    plt.xlabel('Número de Postagens')
    plt.ylabel('Quantidade de Usuários')
    plt.title(titulo)
    plt.show()


plotar_distribuicao(norm_post_count, 'Distribuição de Postagens por Usuário (Normal)')
plotar_distribuicao(alt_post_count, 'Distribuição de Postagens por Usuário (Alta Atividade)')
plotar_distribuicao(baixa_post_count, 'Distribuição de Postagens por Usuário (Baixa Atividade)')

In [ ]:
from utils.estracao_interacao import ExtracaoInteracao

extracao = ExtracaoInteracao(df_normal)
df_with_interactions = extracao.extract_interactions()

# Passo 6: Modificar outras colunas usando .loc para evitar avisos
df_with_interactions['sexo'] = df_with_interactions['sexo'].map({'F': 0, 'M': 1})
df_with_interactions.head()

In [ ]:
from utils.text_treatment import TextTreatment
from utils.busca_palavras import BuscaPalavras

tratamento_texto = TextTreatment()
df_with_interactions['postMessageLimpo'] = df_with_interactions['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')

busca_palavras = BuscaPalavras()

# Faz a busca exata
resultado = busca_palavras.string_matching(df_with_interactions['postMessageLimpo'],
                                           'dados/datasets/termos_depressivos_pt_br.txt')

# Adicionar a coluna quantPalavrasDepressivas ao DataFrame, contando quantas palavras depressivas foram encontradas
df_with_interactions['quantPalavrasDepressivas'] = resultado.apply(lambda x: len(x.split(', ')) if x else 0)
df_with_interactions.head()

In [ ]:
df_with_interactions.sort_values(['id_usuario', 'data'], inplace=True)

# Definindo as variáveis que variam no tempo
variaveis_temporais = ['quantProfile', 'quantCover', 'quantAddPhotoWithOthers',
                       'quantIsWithOthers', 'quantAddPhoto', 'quantSharedPhoto',
                       'quantSharedVideo', 'quantSharedLink', 'quantSharedPost',
                       'quantSharedEvent', 'quantSharedMemory', 'quantStatus',
                       'quantPalavrasDepressivas']

# Primeiro, conte o número de postagens por dia para cada usuário
df_postagens_por_dia = df_with_interactions.groupby(['id_usuario', 'data']).size().reset_index(name='num_postagens')

# Combine os dados de postagens com as outras variáveis temporais
df_grupo = df_with_interactions.groupby(['id_usuario', 'data'])[variaveis_temporais].sum().reset_index()

# Mesclar o DataFrame de postagens com as variáveis temporais
df_grupo = pd.merge(df_grupo, df_postagens_por_dia, on=['id_usuario', 'data'])

janela_temporal = 30


def preparar_serie_temporal(df, janela):
    series_temporais = []
    for usuario in df['id_usuario'].unique():
        df_usuario = df[df['id_usuario'] == usuario].set_index('data')

        # Aplicando rolling nas variáveis temporais
        series_temporais_usuario = df_usuario[variaveis_temporais].rolling(window=janela, min_periods=1).sum()

        # Adicionando a coluna 'num_postagens' ao DataFrame
        series_temporais_usuario['num_postagens'] = df_usuario['num_postagens']

        # Adicionar a coluna 'id_usuario' de volta
        series_temporais_usuario['id_usuario'] = usuario

        # Adiciona ao resultado final
        series_temporais.append(series_temporais_usuario)

    return pd.concat(series_temporais)


# Aplicando a função para criar janelas temporais deslizantes
df_series_temporais = preparar_serie_temporal(df_grupo, janela_temporal).reset_index()

In [ ]:
# Preparação do conjunto de treinamento
X = []
y = []

# Preparando a sequência temporal para treinamento
for usuario in df_series_temporais['id_usuario'].unique():
    df_usuario = df_series_temporais[df_series_temporais['id_usuario'] == usuario]
    for i in range(janela_temporal, len(df_usuario)):
        # X vai conter as variáveis temporais para a janela atual
        X.append(df_usuario.iloc[i - janela_temporal:i][variaveis_temporais].values)

        y.append(df_usuario.iloc[i]['num_postagens'])

X = np.array(X)  # Transformar em array numpy
y = np.array(y)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados de entrada (X)
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Normalizar os dados de saída (y)
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# Criando o modelo LSTM
model = Sequential()

# Primeira camada LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Segunda camada LSTM
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Camada densa final para prever a variável alvo (número de postagens, por exemplo)
model.add(Dense(units=1))  # 1 unidade para previsão de uma variável

# Compilando o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Configurando o Early Stopping para interromper o treinamento quando o val_loss não melhorar
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32,
                    validation_data=(X_test_scaled, y_test_scaled), callbacks=[early_stopping])

# Fazer previsões com os dados de teste normalizados
previsoes_scaled = model.predict(X_test_scaled)

# Desnormalizar as previsões
previsoes = scaler_y.inverse_transform(previsoes_scaled)

# Comparar previsões com os dados reais (y_test)
erro_mse = mean_squared_error(y_test, previsoes)
print(f'Erro MSE: {erro_mse}')

erro_mae = mean_absolute_error(y_test, previsoes)
print(f'Erro MAE: {erro_mae}')

In [ ]:
# Plotar as perdas de treino e validação
plt.plot(history.history['loss'], label='Loss (Treinamento)')
plt.plot(history.history['val_loss'], label='Loss (Validação)')
plt.title('Curva de Perda')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plotar os valores reais vs previsões
plt.scatter(y_test, previsoes)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Previsões vs Valores Reais')
plt.show()


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, previsoes)
print(f'R²: {r2}')
